# RAG using Ollama and Langchain

## Installing required packages

In [ ]:
%pip install langchain_community unstructured unstructured[all-docs] langchain chromadb langchain-text-splitters

Optional installation if you haven't already installed any other version of torch

In [ ]:
# %pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu

##### Along with these packages download and install ollama from - https://ollama.com/download

After installing ollama successfully pull the required models from ollama locally on your PC

In [ ]:
!ollama pull nomic-embed-text
!ollama pull llama3:latest

In [ ]:
!ollama list

## Document Processing

### 1.Loading the document

Importing all the packages used for loading the document

In [1]:
import os
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

Loading the PDF using the "UnstructuredPDFLoader" of "langchain_community"

In [2]:
local_path = "MotorACT.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

d:\LegalBot\RAG_USING_OLLAMA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data[0].page_content

'THE MOTOR VEHICLES ACT, 1988 ____________\n\nARRANGEMENT OF SECTIONS ____________\n\nCHAPTER I PRELIMINARY\n\nSECTIONS\n\n1. Short title, extent and commencement. 2. Definitions. 2A. e-cart and e-rickshaw.\n\nCHAPTER II LICENSING OF DRIVERS OF MOTOR VEHICLES\n\n3. Necessity for driving licence. 4. Age limit in connection with driving of motor vehicles. 5. Responsibility of owners of motor vehicles for contravention of sections 3 and 4. 6. Restrictions on the holding of driving licences. 7. Restrictions on the granting of learner’s licences for certain vehicles. 8. Grant of learner’s licence. 9. Grant of driving licence. 10. Form and contents of licences to drive. 11. Additions to driving licence. 12. Licensing and regulation of schools or establishments for imparting instruction in driving of motor vehicles.\n\n13. Extent of effectiveness of licences, to drive motor vehicles. 14. Currency of licences to drive motor vehicles. 15. Renewal of driving licences. 16. Revocation of driving l

### 3. Converting the document into "chunks"

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

Setting location to save the vector store

In [6]:
current_dir = os.getcwd()
persistent_directory = os.path.join(current_dir, "db", "chroma_db_for_MotorAct")

This is the function that is used to convert the chunks into embeddings

In [7]:
embedding_function = OllamaEmbeddings(model="nomic-embed-text", show_progress=True)

### 4. Creating and storing Embeddings

Here we are Embedding the data and simultaneously storing it in the vector store using the "chromadb"

In [8]:
if os.path.exists(persistent_directory):
    vector_db = Chroma(
        persist_directory=persistent_directory, 
        embedding_function=embedding_function,
        collection_name="local-rag"
    )
    print("Loaded existing Chroma vector store.")
else:
    vector_db = Chroma.from_documents(
        documents=chunks, 
        embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
        collection_name="local-rag",
        persist_directory=persistent_directory
    )
    vector_db.persist()

d:\LegalBot\RAG_USING_OLLAMA\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


Loaded existing Chroma vector store.


## Retrieval And Question Answering

In [9]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In this example we are using the "llama3" model from meta, which is an open-source LLM(Large Language Model)

In [10]:
local_model = "llama3"
llm = ChatOllama(model=local_model)

Here we are giving specific instructions to the bot on how to answer the queries asked by the end users

In [11]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI legal assistant. Your task is to generate five
    different versions of the given legal question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [12]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

template = """You are an AI legal assistant. Answer the question based ONLY on the following legal context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

Here with the help of chaining we are retriving the embeddings from the vector store, converting it to text, passing it to the LLM along with the query all at once

In [13]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Finally we are invoking the chain by taking the query from the user and giving the output

In [ ]:
chain.invoke(input("Ask any question regarding the PDF you uploaded earlier: "))